<a href="https://colab.research.google.com/github/Nargiz-chess/Solar_power_forecasting/blob/main/Moving_average_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pandas import read_csv
import pandas as pd
import datetime as dt
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statistics import mean
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import csv

Mounted at /content/drive


In [ ]:
path1 = '/content/drive/MyDrive/Master_thesis/Datasets/Australia_numpy_removed_night_hours.npy'
path2 = '/content/drive/MyDrive/Master_thesis/Datasets/irradiance_AUS(2012-2013).csv'

#Parameters to change

In [ ]:
test_size = 20
n_house = test_size
n_days = 5

In [ ]:
weath_data=read_csv(path2, header =0, index_col=0) 
weath_data=weath_data.drop("time", axis=1)
weath_data=weath_data.values
weath_data = weath_data.reshape(weath_data.shape[0]*weath_data.shape[1])
weath_data.shape

(6570,)

In [ ]:
np.random.seed(42)

houses = np.loadtxt(path1)
shuffle = np.arange(houses.shape[1])
np.random.shuffle(shuffle)
houses = houses[:,shuffle]
houses.shape

(6570, 299)

In [ ]:
print(shuffle)

[281 265 164   9  77 278  93 109   5 173  97 195 184 154  57  60 147 108
  63 140 155 104 247  46  42 275 280 116 213 236  17 239  33  24  45   7
 113 194 111  92  75  82 118  76 129 197 210 288 219 178 144 186  84 248
 277  73 244  25 209  59   6 183 185 146  30  22 254  56 237 285 126 228
 283 255 158 225  78  66 296 192 181  19 170  79  90 132  72  15  10 157
 224  68 222  37  16 119 268 269  67 101  69  31 172 148 223 114  18 179
  96 165 291  86 246 143 231 124 139 152 204  55 232 182 180 137 251  38
 125 193 167 203 112 264 117 282 273 221 176   2 115 177 175 120 208 259
 127  74  29  83 261 107 287 245 250 294 230  65 196  85 211 159  12  35
  28 142 229 279 168  51  95 206 218  41  89 215 136  26 292 141 198   0
 267 271 100 258 253 171  98  36  61 150 234 200 240  11 295 266  27 242
   4 122  32 202 162 226 256 138  62 135 128 289   8  70 263  64  44 233
 156  40 123 274 216 153  23 260 110  81 207 212  39 238 290 284 199  14
  47  94 262 227 272 201 161  43 217 145 190 220 24

In [ ]:
test_houses = houses[:,-20:]
print("Shape of test houses = ", test_houses.shape)

Shape of test houses =  (6570, 20)


In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
from random import random

n_day =1
time_steps = 18
n_lenght = n_day * time_steps

all_predictions = list()

for house in range(test_houses.shape[1]):
  predictions = []
  data = test_houses[:,house]

  start = 0
  end = n_lenght
  for day in range(0,len(data)-n_lenght,time_steps):
    sample = data[start:end]
    sample = sample.reshape(n_day,time_steps)
    avg = np.average(sample,axis = 0)
    predictions.append(avg)

    with open('/content/drive/MyDrive/Master_thesis/Results/Baseline/Average/Input='+str(n_lenght)+'house_num='+str(house)+'.csv','a') as nd:
      write_n = csv.writer(nd)
      write_n.writerow(avg)
    
    start += time_steps
    end += time_steps
  all_predictions.append(predictions)

np.array(all_predictions).shape

(20, 364, 18)

#Main Evaluation

In [ ]:
time_steps = 18
n_lenght = n_day * time_steps

# substract hours with input length from the beginning of the data
data = test_houses[n_lenght:,:]
from statistics import mean

RMSEs = list()
MAEs=list()

for house in range(data.shape[1]):
  actual_house = data[:,house]
  path ='/content/drive/MyDrive/Master_thesis/Results/Baseline/Average/Input='+str(n_lenght)+'house_num='+str(house)+'.csv'
  pred_house = read_csv(path,header=None)[:-1]
  pred_house = np.array(pred_house)
  #print(actual_house.shape,pred_house.shape)
  
  row_pred = 0
  for hours in range(0,data.shape[0]-n_lenght,18):
    actual =  actual_house[hours: 18+hours]
    pred = pred_house[row_pred,:]
 
    rmse = sqrt(mean_squared_error(actual,pred))
    RMSEs.append(rmse)
    mae= mean_absolute_error(actual,pred) 
    MAEs.append(mae)
    row_pred+=1

print("RMSE",mean(RMSEs),"Shape of RMSEs = ", np.array(RMSEs).shape)
print("MAE",mean(MAEs),"Shape of MAEs = ", np.array(MAEs).shape)

RMSE 0.20752998798833838 Shape of RMSEs =  (7260,)
MAE 0.1260386516681971 Shape of MAEs =  (7260,)
